In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.datasets.samples_generator import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Classificação com NN

Com aquilo que aprendemos, vamos tentar resolver um problema de classificação binária. Veja como é simples apenas modificando um pouco a estrutura de nossa rede.

In [0]:
from sklearn.datasets import make_classification

In [0]:
X, y = make_classification(n_samples = 100000)

In [0]:
X[0]

In [0]:
y[0]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5)

In [0]:
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

In [0]:
X_train[0]

Vamos criar nossa função `build_model` verifique que agora temos um problema de classificação binário, sendo assim, iremos utilizar como função de loss a `binary_crossentropy`.

In [0]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(20, input_dim = 20, activation=tf.nn.relu))
    model.add(keras.layers.Dense(40, activation=tf.nn.relu))
    model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

Verifique que modificamos também nossa função de otimização e métrica. Para entender melhor como o algoritmo de otimização "adam" funciona, veja o seguinte [artigo](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/).

Vamos modificar nossa função para plotar os gráficos

In [0]:
def plot_history(history):
  plt.figure(figsize = (15, 10))
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(history.epoch, np.array(history.history['acc']), label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_acc']), label = 'Val loss')
  plt.legend()

In [0]:
model = build_model()

history = model.fit(X_train, y_train, epochs = 5, validation_split = 0.2, verbose = False)

In [0]:
[loss, acc] = model.evaluate(X_test, y_test)

print("Acc: {:.4f}".format(acc))

In [0]:
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc

Podemos plotar nossa matriz de confusão:

In [0]:
test_preds = model.predict(X_test).flatten()
test_preds_bin = (test_preds > 0.5)
test_preds_bin

## Curva ROC

A Curva de operação do receptor (*Receiver Operationg Characteristic*)  é uma representação gráfica que ilustra o desempenho (ou performance) de um sistema classificador binário e como o seu limiar de discriminação é variado. [Wikipedia](https://pt.wikipedia.org/wiki/Caracter%C3%ADstica_de_Opera%C3%A7%C3%A3o_do_Receptor)

 ![PIC6](https://ncss-wpengine.netdna-ssl.com/wp-content/uploads/2016/06/ROC-Curves-Empirical-19.png)

#### Sensitivity

Também chamada de *true positive rate*, *recall* ou *probabilidade de detecção*. Mede a proporção de positivos realmente detectados como positivos.

$$
TPR = \frac{TP}{TP + FN}
$$

#### Specificity

Também chamada de *true negative rate*, mede a proporção de negativos realmente detectados como negativos.

$$
TNR = \frac{TN}{TN + FP}
$$

In [0]:
YouTubeVideo('21Igj5Pr6u4', width = 720, height = 560)

Obtendo os valores:

In [0]:
fpr, tpr, thresholds = roc_curve(y_test, test_preds)
auc_calc = auc(fpr, tpr)

## Desafio

Utilize o dataset fornecido a tente criar um modelo capaz de classificar os dados não conhecidos pela coluna diagnóstico.

In [0]:
df = pd.read_csv('https://github.com/pgiaeinstein/aula-09/raw/master/df_map.csv')

In [0]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [0]:
notebook_seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = notebook_seed)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Trabalhando com arquivos DICOM

## O que é DICOM?

- DICOM - Digital Imaging and Communications in Medicine;

- Conjunto de normas para tratamento, armazenamento e transmissão de informação médica (PROTOCOLO);
 - Padroniza a formatação das imagens diagnósticas como tomografias, ressonâncias magnéticas, radiografias, ultrassonografias, etc...;
  - Permite que imagens médicas e informações associadas sejam trocadas entre equipamentos de diagnóstico geradores de imagens, computadores e hospitais, ou seja, estabelece uma linguagem comum entre os equipamentos de diferentes marcas.

[Wikipedia](https://pt.wikipedia.org/wiki/DICOM)

## Protocolo?

- Convenção que controla e possibilita uma conexão, comunicação, transferência de dados entre dois sistemas computacionais.

- Define regras de sintaxe, semântica e sincronização da comunicação.

---

| Protocolo | Descrição |
|--|--|
| IP | [Internet Protocol](https://pt.wikipedia.org/wiki/Internet_Protocol "Internet Protocol") |
| HTTP | [Hypertext Transfer Protocol](https://pt.wikipedia.org/wiki/HTTP "HTTP") |
| POP3 | [Post Office Protocol](https://pt.wikipedia.org/wiki/Post_Office_Protocol "Post Office Protocol") |
| SMTP | [Simple Mail Transfer Protocol](https://pt.wikipedia.org/wiki/Simple_Mail_Transfer_Protocol "Simple Mail Transfer Protocol") |


## PyDICOM

In [0]:
!wget "https://github.com/pgiaeinstein/aula-09/raw/master/img1.dcm"
!wget "https://github.com/pgiaeinstein/aula-09/raw/master/img2.dcm"

In [0]:
# É necessário instalar o pacote pydicom pelo gerenciador de pacotes pip
!pip install pydicom

In [0]:
# Após isso, podemos importar o pacote
import pydicom
import pylab

In [0]:
dcmdata = pydicom.read_file('./img1.dcm')
print(dcmdata)

In [0]:
dcmimg = dcmdata.pixel_array
print(dcmimg.dtype)
print(dcmimg.shape)

In [0]:
plt.figure(figsize=(20,10))
plt.imshow(dcmimg, cmap=pylab.cm.binary)
plt.axis('off')

## Desafio

Repita o processo acima para o arquivo img2.dcm